In [1]:
# coding: utf-8
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import nltk
import re
import os
import math
import json
import sys
import time
import json
from PorterStemmer import *
#from nltk.stem import PorterStemmer
#from nltk.tokenize import word_tokenize
ps = PorterStemmer()
from tqdm.notebook import tqdm

In [2]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [3]:
delim = '''[ ',(){}.:;"`\n]'''

In [4]:
data_dir = "C:\Files\col764-a2-release\col764-ass2-release"

In [6]:
metadata = os.path.join(data_dir,"metadata.csv")

In [7]:
meta_df = pd.read_csv(metadata)
meta_df = meta_df[['cord_uid','title','abstract','pdf_json_files','pmc_json_files']]
#meta_df = meta_df.head(1000)
meta_df['pdf_json_files'] = meta_df['pdf_json_files'].astype(str)
meta_df['pmc_json_files'] = meta_df['pmc_json_files'].astype(str)

c:\users\kshitij alwadhi\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
meta_df.head()

,cord_uid,title,abstract,pdf_json_files,pmc_json_files
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json


In [8]:
def getTokensFromText(text):
    words = re.split(delim, text)
    res = []
    for w in words:
        if(len(w) > 0):
            #temp = ps.stem(w.lower(), 0, len(w)-1)
            temp = w.lower()
            if temp not in stop_words:
                res.append(temp)
    return res

In [9]:
def get_idf_dict():
    idf_dict = {}
    filemap = {}
    for i,row in tqdm(meta_df.iterrows()):
        uid = row['cord_uid']
        alltext = ""
        if row['pmc_json_files']:
            paths = row['pmc_json_files']
        elif row['pdf_json_files']:
            paths = row['pdf_json_files']
        filemap[uid] = paths
        for path in paths.split("; "):
            if(path == "nan"):
                continue
            datapath = os.path.join(data_dir,path)
            with open(datapath) as data:
                data_dict = json.load(data)
                for section in data_dict['body_text']:
                    section_text = section['text']
                    alltext+=section_text
                    alltext+=" "
        tokens = getTokensFromText(alltext)
        tokens = set(tokens)
        for token in tokens:
            if token not in idf_dict:
                idf_dict[token]=1
            else:
                idf_dict[token]+=1
    return idf_dict,filemap        

In [10]:
def get_idf_dict_new():
    idf_dict = {}
    filemap = {}
    for row in tqdm(meta_df.itertuples(),total = meta_df.shape[0]):
        uid = row.cord_uid
        alltext = ""
        if row.pmc_json_files:
            paths = row.pmc_json_files
        elif row.pdf_json_files:
            paths = row.pdf_json_files
        filemap[uid] = paths
        for path in paths.split("; "):
            if(path == "nan"):
                continue
            datapath = os.path.join(data_dir,path)
            with open(datapath) as data:
                data_dict = json.load(data)
                for section in data_dict['body_text']:
                    section_text = section['text']
                    alltext+=section_text
                    alltext+=" "
        tokens = getTokensFromText(alltext)
        tokens = set(tokens)
        for token in tokens:
            if token not in idf_dict:
                idf_dict[token]=1
            else:
                idf_dict[token]+=1
    return idf_dict,filemap        

In [20]:
idf_dict,filemap = get_idf_dict_new()

In [54]:
def get_filepath():
    filemap = {}
    meta_df2 = pd.read_csv(metadata)
    meta_df2 = meta_df2[['cord_uid','title','abstract','pdf_json_files','pmc_json_files']]
    meta_df2['pdf_json_files'] = meta_df2['pdf_json_files'].astype(str)
    meta_df2['pmc_json_files'] = meta_df2['pmc_json_files'].astype(str)
    for row in tqdm(meta_df2.itertuples(),total = meta_df2.shape[0]):
        uid = row.cord_uid
        alltext = ""
        if row.pmc_json_files:
            paths = row.pmc_json_files
        elif row.pdf_json_files:
            paths = row.pdf_json_files
        filemap[uid] = paths
    return filemap        
filemap = get_filepath()

In [21]:
def dumpdicts(idf_dict,filemap):
    with open("idf_dict.json", "w") as outfile:
        json.dump(idf_dict, outfile)
    with open("filemap.json", "w") as outfile:
        json.dump(filemap, outfile)
dumpdicts(idf_dict,filemap)

In [11]:
with open("idf_dict.json", "r") as jsonfile:
    idf_dict = json.load(jsonfile)
with open("filemap.json", "r") as jsonfile:
    filemap = json.load(jsonfile)

In [12]:
def create_vocab_dict(idf_dict):
    vocab = {}
    for i,word in enumerate(idf_dict.keys()):
        vocab[word] = i
    return vocab

In [13]:
vocab = create_vocab_dict(idf_dict)

In [14]:
vocab

{'tertiary': 0,
 'three': 1,
 'systolic': 2,
 'hemorrhages': 3,
 'light': 4,
 'include': 5,
 '24': 6,
 '567': 7,
 'system': 8,
 'comparable': 9,
 'manufacturer': 10,
 '11': 11,
 'musculoskeletal': 12,
 'beyond': 13,
 'evan': 14,
 'cytomegalo': 15,
 'outpatient': 16,
 'occur': 17,
 'departments': 18,
 '>140': 19,
 '<90': 20,
 '22': 21,
 '16': 22,
 'protean': 23,
 'and/or': 24,
 'nosocomial': 25,
 'accessed': 26,
 'moderate': 27,
 'identification': 28,
 'clinical': 29,
 '40%': 30,
 'annual': 31,
 'psychosis': 32,
 'edema': 33,
 'white': 34,
 'mild': 35,
 'agars': 36,
 '4': 37,
 '60': 38,
 'tachypnoea': 39,
 '<8': 40,
 'sick': 41,
 'kpa': 42,
 '20/36': 43,
 'blood': 44,
 'populations': 45,
 'effusion': 46,
 '5]': 47,
 'bias': 48,
 'instructions': 49,
 '16/31': 50,
 '87': 51,
 'clinically': 52,
 'thirty-one': 53,
 'singly': 54,
 'former': 55,
 'gram-stained': 56,
 'chi-square': 57,
 'program': 58,
 '7%': 59,
 'contact': 60,
 'fermentation': 61,
 'adults': 62,
 'also': 63,
 'muscle': 64,
 '

In [15]:
def get_tf_dict(docid):
    tf_dict = {}
    paths = filemap[docid]
    alltext = ""
    for path in paths.split("; "):
        if(path == "nan"):
            continue
        datapath = os.path.join(data_dir,path)
        with open(datapath) as data:
            data_dict = json.load(data)
            for section in data_dict['body_text']:
                section_text = section['text']
                alltext+=section_text
                alltext+=" "
    tokens = getTokensFromText(alltext)
    for token in tokens:
        if token not in tf_dict:
            tf_dict[token]=1
        else:
            tf_dict[token]+=1
    return tf_dict

In [16]:
N = len(filemap)

In [17]:
def get_idf(term):
    return math.log(1+N/idf_dict[term],2)

In [18]:
def getDocVector(docid):
    tf_dict = get_tf_dict(docid)
    vec = np.zeros(len(vocab))
    for word,tf in tf_dict.items():
        if word in vocab:
            pos = vocab[word]
            normtf = 1 + math.log(tf,2)
            normidf = get_idf(word)
            tfidf = normtf * normidf
            vec[pos] = tfidf
    return vec

In [29]:
#getDocVector('ug7v899j')

In [19]:
def getQueryVector(query):
    tokens = getTokensFromText(query)
    query_tf_dict = {}
    for token in tokens:
        if token not in query_tf_dict:
            query_tf_dict[token]=1
        else:
            query_tf_dict[token]+=1
    vec = np.zeros(len(vocab))
    for word,tf in query_tf_dict.items():
        if word in vocab.keys():
            pos = vocab[word]
            normtf = 1 + math.log(tf,2)
            normidf = get_idf(word)
            tfidf = normtf * normidf
            vec[pos] = tfidf
    return vec

In [20]:
temp = getQueryVector("this is a sample query")
for e in temp:
    if e >0:
        print(e)

3.299325612080061
7.153821821804281


In [21]:
def getSim(queryVec,docVec):
    if np.linalg.norm(queryVec) == 0 or np.linalg.norm(docVec)==0:
        return 0.0
    return np.dot(queryVec,docVec)/(np.linalg.norm(queryVec) * np.linalg.norm(docVec))

In [22]:
getSim(getQueryVector("pneumonia disease breathe"),getDocVector('ug7v899j'))

0.04362011323686911

In [23]:
getSim(getQueryVector("inflammatory diseases"),getDocVector('02tnwd4m'))

0.07223477347244252

In [24]:
getSim(getQueryVector("disease"),getDocVector('02tnwd4m'))

0.03594977410131264

In [25]:
topics_file = "covid19-topics.xml"
top100_file= "t40-top-100.txt"
relevance_file = "t40-qrels.txt"

In [26]:
def get_queries():
    with open(os.path.join(data_dir,topics_file), "r") as file:
        content = file.readlines()
        content = "".join(content)
        bs_content = BeautifulSoup(content, "lxml")
        queries = bs_content.find_all("query")
        all_text = []
        for q in queries:
            all_text.append(q.text)
        return all_text

In [27]:
def get_relevances():
    rel_dict = {}
    with open(os.path.join(data_dir,relevance_file),"r") as file:
        for x in file:
            line = x.split(" ")
            qid = int(line[0])
            docid = line[2]
            rel = int(line[3].replace("\n",""))
            if qid not in rel_dict:
                rel_dict[qid] = {docid:rel}
            else:
                rel_dict[qid][docid] = rel
    return rel_dict

In [28]:
def get_top100():
    top100_dict = {}
    with open(os.path.join(data_dir,top100_file),"r") as file:
        for x in file:
            line = x.split(" ")
            if(len(line)>1):
                qid = int(line[0])
                docid = line[2]
                if qid not in top100_dict:
                    top100_dict[qid] = [docid]
                else:
                    top100_dict[qid].append(docid)
    return top100_dict

In [29]:
top100_dict = get_top100()
rel_dict = get_relevances()

In [30]:
def test():
    docs = top100_dict[1]
    for doc in docs:
        if doc in rel_dict[1]:
            print(rel_dict[1][doc])
        else:
            print(0)

In [39]:
def getNDCG(qno,docs):
    def discount_helper(arr):
        score = 0
        for i,e in enumerate(arr):
            score += e/math.log(i+2,2)
        return score
    scores = []
    for doc in docs:
        if doc in rel_dict[qno]:
            scores.append(rel_dict[qno][doc])
        else:
            scores.append(0)
    weighted_scores = []
    for score in scores:
        if score==0:
            weighted_scores.append(5)
        elif score==1:
            weighted_scores.append(10)
        elif score==2:
            weighted_scores.append(50)
    scores = weighted_scores
    origscore = discount_helper(scores)
    sortedscore = discount_helper(sorted(scores,reverse=True))
    if sortedscore == 0:
        return 0
    return origscore/sortedscore

In [40]:
getNDCG(1,top100_dict[1])

0.7581423229789526

In [54]:
def getAvgNDCG():
    total = 0
    all_scores = []
    for i in range(len(top100_dict)):
        qno = i+1
        qndcg = getNDCG(qno,top100_dict[qno])
        all_scores.append(qndcg)
        total +=qndcg
    return all_scores,total/len(top100_dict)

In [55]:
getAvgNDCG()

([0.7581423229789526,
  0.6449764123560574,
  0.6416177154025472,
  0.737695467579644,
  0.6017951740123688,
  0.9214352018876463,
  0.732061319618504,
  0.6282502836158507,
  1.0,
  0.7871882647938129,
  0.590973317561294,
  0.7013811884718257,
  0.7539044592944869,
  0.7879971801679476,
  0.7987027996272297,
  0.6793359643226071,
  0.8821922854723496,
  0.908242656284347,
  0.6798010744016668,
  0.850212281441203,
  0.8615603904908614,
  0.7821691017370824,
  0.9125155396077322,
  0.9713056818671552,
  0.8442448341411152,
  0.8223730394472976,
  0.9140791821881469,
  0.9769027979371936,
  0.8028143570119931,
  0.9337953214232392,
  0.5921814495317327,
  0.6512846092892439,
  0.8845668729184191,
  0.6160796792948718,
  0.8020921395255738,
  0.9867351807929213,
  0.8214683705576603,
  0.9404310949104868,
  0.9511969335278158,
  0.8759360537089681],
 0.800740949979996)

In [35]:
def rocchio_updateQueries(alpha,beta,gamma):
    updated_queries = []
    for i,q_orig in tqdm(enumerate(get_queries())):
        q = q_orig.lower()
        qv0 = getQueryVector(q)
        qno = i+1
        top100docs = top100_dict[qno]
        relvec = np.zeros(len(vocab))
        irrelvec = np.zeros(len(vocab))
        rel_vecarr = []
        irrel_vecarr = []
        for doc in top100docs:
            if doc in rel_dict[qno]:
                rel = rel_dict[qno][doc]
            else:
                rel = 0
            if rel >0:
                rel_vecarr.append(getDocVector(doc))
            else:
                irrel_vecarr.append(getDocVector(doc))
        for v in rel_vecarr:
            relvec +=v
        for v in irrel_vecarr:
            irrelvec +=v
        if len(rel_vecarr)>0:
            relvec /= len(rel_vecarr)
        if len(irrel_vecarr)>0:
            irrelvec /= len(irrel_vecarr)
        qnew = np.multiply(qv0,alpha) +np.multiply(relvec,beta) - np.multiply(irrelvec,gamma)
        qnew = np.where(qnew<0, 0, qnew)
        updated_queries.append(qnew)
    return updated_queries

In [36]:
updated_queries = rocchio_updateQueries(1.0,0.75,0.15)

In [44]:
def rocchio_getNDCG(updated_queries):
    total = 0
    allscores = []
    for i,q in tqdm(enumerate(updated_queries)):
        qno = i+1
        doc_score_dict = {}
        top100docs = top100_dict[qno]
        for doc in top100docs:
            score = getSim(q,getDocVector(doc))
            doc_score_dict[doc] = score
        sorted_docs = dict(sorted(doc_score_dict.items(), key=lambda x: x[1],reverse = True))
        sorted_docs = list(sorted_docs.keys())
        ndcg = getNDCG(qno,sorted_docs)
        allscores.append(ndcg)
        total +=ndcg
    return allscores,total/len(updated_queries)

In [43]:
rocchio_getNDCG(updated_queries)

0.8635512790394195

In [58]:
def gridSearch():
    res = {'alpha':[],'beta':[],'gamma':[],'qid':[],'nDCG':[]}
    avgres = {'alpha':[],'beta':[],'gamma':[],'nDCG':[]}
    df = pd.DataFrame(res)
    avgdf = pd.DataFrame(avgres)
    for alpha in [1.0]:
        for beta in [0.75]:
            for gamma in [0.15]:
                updated_queries = rocchio_updateQueries(alpha,beta,gamma)
                allscores,ndcg = rocchio_getNDCG(updated_queries)
                alpha_arr = [alpha] * len(updated_queries)
                beta_arr = [beta] * len(updated_queries)
                gamma_arr = [gamma] * len(updated_queries)
                q_arr = np.arange(1,len(updated_queries)+1,1)
                df2 = pd.DataFrame({'alpha':alpha_arr,'beta':beta_arr,'gamma':gamma_arr,'qid':q_arr,'nDCG':allscores})
                df = df.append(df2,ignore_index=True)
                avgdf = avgdf.append(pd.DataFrame({'alpha':[alpha],'beta':[beta],'gamma':[gamma],'nDCG':[ndcg]}))
    df.to_csv('query_wise.csv',index=False)
    avgdf.to_csv('avg.csv',index=False)

In [59]:
gridSearch()